# Difinity 2025 - Demo Notebook

## 1. Install Required Libraries

Install Semantic Link Labs to enable Direct Lake model creation and management capabilities.

In [ ]:
%pip install -q --upgrade pip
%pip install -q semantic-link-labs

## 2. Import Libraries and Set Variables

Import required Python libraries and define key variables for the lakehouse and semantic model names.

In [5]:
import sempy_labs as labs
from sempy import fabric
import sempy
import pandas
import json
import time

fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM
import Microsoft.AnalysisServices
import warnings
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable

LakehouseName = "AdventureWorks"
SemanticModelName = f"{LakehouseName}_model"

StatementMeta(, e51a042a-dda0-41d8-9fbb-3a447d044926, 14, Finished, Available, Finished)

## 3. Create or Connect to Lakehouse

Check if the AdventureWorks lakehouse exists, create it if needed, and retrieve workspace identifiers.

In [ ]:
lakehouses=labs.list_lakehouses()["Lakehouse Name"]
if LakehouseName in lakehouses.values:
    lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]
else:
    lakehouseId = fabric.create_lakehouse(LakehouseName)

workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
workspaceName = sempy.fabric.resolve_workspace_name(workspaceId)
print(f"WorkspaceId = {workspaceId}, LakehouseID = {lakehouseId}, Workspace Name = {workspaceName}")

## 4. Load Adventure Works Sample Data

Load four Adventure Works tables (Customer, Date, Product, Sales) into the lakehouse using region-aware data sources.

**Tables being loaded:**
- DimCustomer (~18K customers)  
- DimDate (2K+ dates)
- DimProduct (~600 products)
- FactInternetSales (~60K sales records)

### Behind the Scenes
- Data is stored in **Delta format** 
- **Overwrite mode** ensures clean data for the workshop
- **OneLake integration** provides seamless cross-workspace data access

🎯 **Success indicator**: All four "Loaded" messages followed by "Done"

In [ ]:
capacity_name = labs.get_capacity_name()

def loadDataToLakehouse(fromTable: str, toTable: str):
    """
    Optimized data loading function with improved error handling and performance.
    
    Args:
        fromTable: Source table name to read from
        toTable: Target table name to write to
    """
    try:
        # Get lakehouse properties once and reuse
        lakehouse_props = notebookutils.lakehouse.getWithProperties(LakehouseName)
        workspaceId = lakehouse_props["workspaceId"]
        lakehouseId = lakehouse_props["id"]

        # Region-aware connection string selection
        if capacity_name == "FabConUS8-P1":  # West US 3
            conn_str = "abfss://b1d61bbe-de20-4d3a-8075-b8e2eaacb868@onelake.dfs.fabric.microsoft.com/631e45c0-1243-4f42-920a-56bfe6ecdd6d/Tables"
        else:  # North Central US (default)
            conn_str = "abfss://16cf855f-3bf4-4312-a7a1-ccf5cb6a0121@onelake.dfs.fabric.microsoft.com/99ed86df-13d1-4008-a7f6-5768e53f4f85/Tables"

        # Read source data with format specification for better performance
        customer_df = spark.read.format("delta").load(f"{conn_str}/{fromTable}")
        
        # Cache the DataFrame if it will be used multiple times or is computation-heavy
        customer_df.cache()
        
        # Write with optimized settings
        (customer_df
         .write
         .format("delta")
         .mode("overwrite")
         .option("overwriteSchema", "true")
         .save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/{toTable}"))
        
        # Unpersist cached DataFrame to free memory
        customer_df.unpersist()
        
        print(f"Loaded {toTable}")
        
    except Exception as e:
        print(f"Error loading {toTable}: {e}")
        raise

# Load all tables with proper error handling
tables_to_load = [
    ("DimCustomer", "DimCustomer"),
    ("DimDate", "DimDate"),
    ("DimProduct", "DimProduct"),
    ("FactInternetSales", "FactInternetSales")
]

for from_table, to_table in tables_to_load:
    loadDataToLakehouse(from_table, to_table)

print("Done")

## 5. Trigger Metadata Synchronization

Force synchronization between lakehouse storage and SQL Analytics Endpoint to ensure schema accuracy for the semantic model.

In [ ]:
##https://medium.com/@sqltidy/delays-in-the-automatically-generated-schema-in-the-sql-analytics-endpoint-of-the-lakehouse-b01c7633035d

def triggerMetadataRefresh():
    client = fabric.FabricRestClient()
    response = client.get(f"/v1/workspaces/{workspaceId}/lakehouses/{lakehouseId}")
    sqlendpoint = response.json()['properties']['sqlEndpointProperties']['id']

    # trigger sync
    uri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}"
    payload = {"commands":[{"$type":"MetadataRefreshExternalCommand"}]}
    response = client.post(uri,json= payload)
    batchId = response.json()['batchId']

    # Monitor Progress
    statusuri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}/batches/{batchId}"
    statusresponsedata = client.get(statusuri).json()
    progressState = statusresponsedata['progressState']
    print(f"Metadata refresh : {progressState}")
    while progressState != "success":
        statusuri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}/batches/{batchId}"
        statusresponsedata = client.get(statusuri).json()
        progressState = statusresponsedata['progressState']
        print(f"Metadata refresh : {progressState}")
        time.sleep(1)

    print('Metadata refresh complete')

triggerMetadataRefresh()

## 6. Create Direct Lake Semantic Model

Generate a new Direct Lake semantic model (DL\SQL) from lakehouse tables.

In [ ]:
from sempy import fabric

#1. Generate list of ALL table names from lakehouse to add to Semantic Model
lakehouseTables:list = labs.lakehouse.get_lakehouse_tables(lakehouse=LakehouseName)["Table Name"]

completedOK:bool=False
while not completedOK:
    try:
        #2 Create the semantic model
        if sempy.fabric.list_items().query(f"`Display Name`=='{LakehouseName}_model' & Type=='SemanticModel'  ").shape[0] ==0:
            labs.directlake.generate_direct_lake_semantic_model(dataset=f"{LakehouseName}_model",lakehouse_tables=lakehouseTables,workspace=workspaceName,lakehouse=lakehouseId,refresh=False,overwrite=True)
            completedOK=True
    except:
        print('Error creating model... trying again.')
        time.sleep(3)
        triggerMetadataRefresh()

print('Semantic model created OK')

## 7. Configure Table Relationships

Add relationships to model.

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
            #1. Remove any existing relationships
            for r in tom.model.Relationships:
                tom.model.Relationships.Remove(r)

            #2. Creates correct relationships
            tom.add_relationship(from_table="FactInternetSales", from_column="OrderDateKey" , to_table="DimDate"    , to_column="DateKey"       , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="FactInternetSales", from_column="CustomerKey"  , to_table="DimCustomer", to_column="CustomerKey"   , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="FactInternetSales", from_column="ProductKey"   , to_table="DimProduct" , to_column="ProductKey"    , from_cardinality="Many" , to_cardinality="One")
            completedOK=True
    except:
        print('Error adding relationships... trying again.')
        time.sleep(3)

print('done')


## 8. Add Business Intelligence Measures

Add some basic measures for use in DAX query later in lab.

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
            #1. Remove any existing measures
            for t in tom.model.Tables:
                for m in t.Measures:
                    tom.remove_object(m)
                    print(f"[{m.Name}] measure removed")

            tom.add_measure(table_name="FactInternetSales" ,measure_name="Sum of Sales",expression="SUM(FactInternetSales[SalesAmount])",format_string="\$#,0.###############;(\$#,0.###############);\$#,0.###############")
            tom.add_measure(table_name="FactInternetSales" ,measure_name="Count of Sales",expression="COUNTROWS(FactInternetSales)",format_string="#,0")
            completedOK=True
    except:
        print('Error adding measures... trying again.')
        time.sleep(3)

print('done')

## 9. Configure Date Table for Time Intelligence

Marks DimDate table as date table to enable time-based analysis functions and calendar features.

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
            tom.mark_as_date_table(table_name="DimDate",column_name="Date")
            completedOK=True
    except:
        print('Error with date table... trying again.')
        time.sleep(3)

print('done')

## 10. Configure Column Sorting for Improved User Experience

Sets logical column sorting on date table columns to ensure proper chronological ordering in visualizations.

In [ ]:
import json
tom = labs.tom.TOMWrapper(dataset=SemanticModelName, workspace=workspaceName, readonly=False)
tom.set_sort_by_column(table_name="DimDate",column_name="MonthName"       ,sort_by_column="MonthNumberOfYear")
tom.set_sort_by_column(table_name="DimDate",column_name="DayOfWeek"       ,sort_by_column="DayNumberOfWeek")
tom.model.SaveChanges()

i:int=0
for t in tom.model.Tables:
    if t.Name=="DimDate":
        bim = json.dumps(tom.get_bim()["model"]["tables"][i],indent=4)
        print(bim)
    i=i+1

## 11. Hide Fact Table Columns for Optimal User Experience

Hides raw fact table columns to guide users toward proper measures and improve usability.

In [ ]:
i:int=0
for t in tom.model.Tables:
    if t.Name in ["FactInternetSales"]:
        for c in t.Columns:
            c.IsHidden=True

        bim = json.dumps(tom.get_bim()["model"]["tables"][i],indent=4)
        print(bim)
    i=i+1
    
tom.model.SaveChanges()

## 12. Frame Model to be ready for queries

Frames the semantic model for readiness.

In [ ]:
reframeOK:bool=False
while not reframeOK:
    try:
        result:pandas.DataFrame = labs.refresh_semantic_model(dataset=SemanticModelName)
        reframeOK=True
    except:
        print('Error with reframe... trying again.')
        triggerMetadataRefresh()
        time.sleep(3)

print('Custom Semantic Model reframe OK')

## 13. Setup DMV Monitoring Function

Creates monitoring function to track Direct Lake column temperature and memory usage using DMV queries.

In [6]:
import warnings
import time
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable

def runDMV():
    df = sempy.fabric.evaluate_dax(
        dataset=SemanticModelName, 
        dax_string="""
        
        SELECT 
            MEASURE_GROUP_NAME AS [TABLE],
            ATTRIBUTE_NAME AS [COLUMN],
            DATATYPE ,
            DICTIONARY_SIZE 		    AS SIZE ,
            DICTIONARY_ISPAGEABLE 		AS PAGEABLE ,
            DICTIONARY_ISRESIDENT		AS RESIDENT ,
            DICTIONARY_TEMPERATURE		AS TEMPERATURE,
            DICTIONARY_LAST_ACCESSED	AS LASTACCESSED 
        FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
        ORDER BY 
            [DICTIONARY_TEMPERATURE] DESC
        
        """)
    display(df)

StatementMeta(, e51a042a-dda0-41d8-9fbb-3a447d044926, 15, Finished, Available, Finished)

In [8]:
def filter_func(e):
    retVal:bool=True
    if e.EventSubclass.ToString() == "VertiPaqScanInternal":
        retVal=False      
    #     #if e.EventSubClass.ToString() == "VertiPaqScanInternal":
    #     retVal=False
    return retVal

# define events to trace and their corresponding columns
def runQueryWithTrace (expr:str,workspaceName:str,SemanticModelName:str,Result:Optional[bool]=True,Trace:Optional[bool]=True,DMV:Optional[bool]=True,ClearCache:Optional[bool]=True) -> pandas.DataFrame :
    event_schema = fabric.Trace.get_default_query_trace_schema()
    event_schema.update({"ExecutionMetrics":["EventClass","TextData"]})
    del event_schema['VertiPaqSEQueryBegin']
    del event_schema['VertiPaqSEQueryCacheMatch']
    del event_schema['DirectQueryBegin']

    warnings.filterwarnings("ignore")

    WorkspaceName = workspaceName
    SemanticModelName = SemanticModelName

    if ClearCache:
        labs.clear_cache(SemanticModelName)

    with fabric.create_trace_connection(SemanticModelName,WorkspaceName) as trace_connection:
        # create trace on server with specified events
        with trace_connection.create_trace(
            event_schema=event_schema, 
            name="Simple Query Trace",
            filter_predicate=filter_func,
            stop_event="QueryEnd"
            ) as trace:

            trace.start()

            df=sempy.fabric.evaluate_dax(
                dataset=SemanticModelName, 
                dax_string=expr)

            if Result:
                displayHTML(f"<H2>####### DAX QUERY RESULT #######</H2>")
                display(df)

            # Wait 5 seconds for trace data to arrive
            time.sleep(5)

            # stop Trace and collect logs
            final_trace_logs = trace.stop()

    if Trace:
        displayHTML(f"<H2>####### SERVER TIMINGS #######</H2>")
        display(final_trace_logs)
    
    if DMV:
        displayHTML(f"<H2>####### SHOW DMV RESULTS #######</H2>")
        runDMV()
    
    return final_trace_logs

StatementMeta(, e51a042a-dda0-41d8-9fbb-3a447d044926, 17, Finished, Available, Finished)

## 14. Explore Direct Lake Capabilities with DAX Functions

Uses TABLETRAITS() and guardrails functions to validate Direct Lake configuration and performance limits.

In [ ]:
df=sempy.fabric.evaluate_dax(
    dataset=SemanticModelName, 
    dax_string="""
    
    evaluate tabletraits()
    
    """)
display(df)

In [ ]:
df=labs.directlake.get_direct_lake_guardrails()
display(df)

## 15. Establish Performance Baseline with DMV Analysis

Captures initial Direct Lake column states and memory usage to establish baseline.

In [ ]:
runDMV()

## 16. Execute DAX Query and Monitor Column Loading

Executes DAX query and monitors which columns get loaded into memory using DMV analysis.

In [ ]:
labs.clear_cache(SemanticModelName)

df=sempy.fabric.evaluate_dax(
    dataset=SemanticModelName, 
    dax_string="""
    
    EVALUATE
        SUMMARIZECOLUMNS(
               
                DimDate[MonthName] ,
                "Count of Transactions" , COUNTROWS(FactInternetSales) ,
                "Sum of Sales" , [Sum of Sales] 
        )
        ORDER BY [MonthName]
    """)
display(df)

runDMV()

## 17. Create Report
Create a simple report linked to our semantic model

In [ ]:
import sempy_labs as labs
from sempy_labs import report
import time
import pandas

# Import specialized helper functions for Power BI operations and framing analysis
from sempy_labs._helper_functions import (
    resolve_report_id,
    format_dax_object_name,
    resolve_dataset_from_report,
    _conv_b64,
    _extract_json,
    _add_part,
    _decode_b64,
    resolve_workspace_name_and_id,
    _update_dataframe_datatypes,
    _base_api,
    _create_dataframe,
)

ModuleNotFoundError: No module named 'sempy_labs'

In [ ]:
report_name="Simple Report"

pbi_report:dict = {}
pbi_report['config'] = """{
        "version": "5.37",
        "themeCollection": {},
        "activeSectionIndex": 0,
        "linguisticSchemaSyncVersion": 0,
        "objects": {
            "outspacePane": [
                {
                    "properties": {
                        "expanded": {
                            "expr": {
                                "Literal": {
                                    "Value": "false"
                                }
                            }
                        }
                    }
                }
            ]
        }
    }"""
pbi_report['layoutOptimization']=0
pbi_report['resourcePackages'] = [{'resourcePackage': {'disabled': False, 'items': [{'name': 'CY24SU10', 'path': 'BaseThemes/CY24SU10.json', 'type': 202}], 'name': 'SharedResources', 'type': 2}}]
pbi_report['sections'] = [
    {'config': '{}', 
    'displayName': 'Page 1', 
    'displayOption': 1, 
    'filters': '[]', 
    'height': 300.0, 
    'width': 600.0,
    'name': 'a4c1ed461808909ae820', 
    'visualContainers':
        [
            {'config': '''{
                        "name": "Matrix",
                        "layouts": [
                            {
                                "id": 0,
                                "position": {
                                    "x": 310,
                                    "y": 30,
                                    "z": 1000,
                                    "width": 253,
                                    "height": 202
                                }
                            }
                        ],
                        "singleVisual": {
                            "visualType": "tableEx",
                            "projections": {
                                "Values": [
                                    {
                                        "queryRef": "DimDate.Month"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.SalesAmount)"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.DiscountAmount)"
                                    }
                                ]
                            },
                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "d",
                                        "Entity": "DimDate",
                                        "Type": 0
                                    },
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "d"
                                                }
                                            },
                                            "Property": "Month"
                                        },
                                        "Name": "DimDate.Month",
                                        "NativeReferenceName": "Month"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "SalesAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.SalesAmount)",
                                        "NativeReferenceName": "Sum of SalesAmount"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "DiscountAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.DiscountAmount)",
                                        "NativeReferenceName": "Discount"
                                    }
                                ]
                            },
                            "columnProperties": {
                                "Sum(FactInternetSales.DiscountAmount)": {
                                    "displayName": "Discount"
                                }
                            },
                            "drillFilterOtherVisuals": true,
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    }''', 'filters': '[]', 'height': 202.46, 'width': 215.11, 'x': 319.67, 'y': 30.63, 'z': 1.0
                },
            {'config': 
                    '''{
                        "name":"Card",
                        "layouts":[
                            {
                            "id":0,
                            "position":{"x":10,"y":30,"z":0,"width":238,"height":201}}
                            ],
                        "singleVisual":{"visualType":"card","projections":{"Values":[{"queryRef":"FactInternetSales.Count of Sales"}]},


                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Measure": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "f"
                                                }
                                            },
                                            "Property": "Count of Sales"
                                        },
                                        "Name": "FactInternetSales.Count of Sales",
                                        "NativeReferenceName": "Count of Sales"
                                    }
                                ],
                                "OrderBy": [
                                    {
                                        "Direction": 2,
                                        "Expression": {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {
                                                        "Source": "f"
                                                    }
                                                },
                                                "Property": "Count of Sales"
                                            }
                                        }
                                    }
                                ]
                            },



                            "drillFilterOtherVisuals":true,
                            "hasDefaultSort":true,                           
                            "objects": {
                                "labels": [
                                    {
                                        "properties": {
                                            "fontSize": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "20D"
                                                    }
                                                }
                                            },
                                            "labelDisplayUnits": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "1D"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            },
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                            }
                        }''',
                 'filters': '[]', 
                 'height': 201.5, 
                 'width': 265.43, 
                 'x': 270.03, 
                 'y': 30.12, 
                 'z': 1000.0
            }
        ]
    }]

labs.report.create_report_from_reportjson(report=report_name , dataset="AdventureWorks_model" , report_json = pbi_report)
report_id = resolve_report_id(report_name)

from powerbiclient import Report
simple_report = Report(group_id=None, report_id=report_id)
simple_report.set_size(400,700)
simple_report

## 18. Clone Model

In [25]:
import uuid
fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM
import Microsoft.AnalysisServices
import warnings
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable

StatementMeta(, 045cc713-9995-424a-b100-f258bfbce8d2, 38, Finished, Available, Finished)

In [26]:


ClonedModelName = SemanticModelName + "_clone"

#Clear any existing cloned model if re-running
df = fabric.list_items()
if ClonedModelName in df.values:
    model_id = df.at[df[df['Display Name'] == ClonedModelName].index[0], 'Id']
    fabric.delete_item(model_id)
    print("Cloned model deleted")

with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
    newDB = tom._tom_server.Databases.GetByName(SemanticModelName).Clone()
    newModel = tom._tom_server.Databases.GetByName(SemanticModelName).Model.Clone()
    newDB.Name = ClonedModelName
    newDB.ID = str(uuid.uuid4())
    #newDB.Model = newModel
    newModel.CopyTo(newDB.Model)
    tom._tom_server.Databases.Add(newDB)

    newDB.Update(Microsoft.AnalysisServices.UpdateOptions.ExpandFull)

StatementMeta(, 045cc713-9995-424a-b100-f258bfbce8d2, 39, Finished, Available, Finished)

## Run Query with Trace

In [ ]:
workspace_name = None #"Difinity 2025 Notebooks"
df = runQueryWithTrace("""
        EVALUATE
	        SUMMARIZECOLUMNS(
		        DimDate[DateKey],
		        "Quantity" , [Sum of Sales],
                "YTD Orders", TotalYTD(SUM(FactInternetSales[OrderQuantity]), DimDate[Date])
		        )
""",workspace_name,SemanticModelName,DMV=False)

display(df)

## Run Load Test
https://github.com/microsoft/fabric-toolbox/tree/main/tools/FabricLoadTestTool

## 19. Semantic Link Labs demos
https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples

### 19.1 BPA
Run the Best Practice Analyzer

In [ ]:
import sempy_labs as labs

workspace = None # Enter the name or ID of the workspace in which the semantic model resides

labs.run_model_bpa(dataset=SemanticModelName, workspace=workspace)
#labs.run_model_bpa(dataset=dataset, workspace=workspace, extended=True) # Setting extended=True will fetch Vertipaq Analyzer statistics and use them to run advanced BPA rules against your model
# labs.run_model_bpa(dataset=dataset, workspace=workspace, export=True) # Setting export=True will export the results to a delta table in the lakehouse attached to the notebook
# labs.run_model_bpa(dataset=dataset, workspace=workspace, language="French") # Setting the 'language' parameter will dynamically translate the rules, categories and descriptions to the specified language.

### 19.2 Vertipaq Analyzer
Run Vertipaq Analyzer

In [ ]:
import sempy_labs as labs


x = labs.vertipaq_analyzer(dataset=SemanticModelName, workspace=workspace)
#x = labs.vertipaq_analyzer(dataset=dataset, workspace=workspace, export='table') # Setting export='table' will export the results to delta tables in the lakehouse attached to the notebook
#x = labs.vertipaq_analyzer(dataset=dataset, workspace=workspace, export='zip') # Setting export='zip' will export the results to a .zip file in the lakehouse attached to the notebook.

# Note that this function returns a dictionary of dataframes which is captured above as the parameter 'x'. You can view this as such:
#for name, df in x.items():
#    print(name)
#    display(df)

### Generate .vpax file

In [ ]:
import sempy_labs as labs

dataset = 'AdventureWorks_model' # Enter the name or ID of your semantic model
workspace = None #'3a77825c-982f-42f0-99dd-b0c0c1a9d3f5' # Enter the name or ID of the workspace in which the semantic model resides
lakehouse = 'vpax'
lakehouse_workspace = 'dg_test'

labs.create_vpax(
    dataset=SemanticModelName,
    workspace=workspace,
    read_stats_from_data=True,
    lakehouse=lakehouse,
    lakehouse_workspace=lakehouse_workspace,
    overwrite=True,
    direct_lake_stats_mode='Resident'
)

## 20. Clean Up Resources and Session Conclusion

In [ ]:
mssparkutils.session.stop()